# 1단계: 노트북에서 실험
<div class="alert alert-warning"> 이 노트북은 <code>SageMaker Distribution Image 3.6.1</code>을 사용하는 SageMaker Studio JupyterLab 인스턴스에서 SageMaker Python SDK 버전 <code>2.255.0</code>으로 마지막으로 테스트
되었습니다</div>

이 단계에서는 노트북에서 로컬로 데이터 처리와 모델 훈련 및 평가를 실행합니다. `sagemaker` 또는 `boto3` 패키지를 사용하지 않습니다.

아이디어에서 프로덕션까지 6단계:
||||
|---|---|---|
|1. |노트북에서 실험 |**<<<< 현재 위치**|
|2. |SageMaker AI 처리 작업과 SageMaker SDK로 확장 ||
|3. |ML 파이프라인, 모델 레지스트리, 피처 스토어로 운영화 ||
|4. |모델 빌드 CI/CD 파이프라인 추가 ||
|5. |모델 배포 파이프라인 추가 ||
|6. |모델 및 데이터 모니터링 추가 ||

<div class="alert alert-info"> 이 노트북에서는 JupyterLab에서 <code>Python 3</code> 커널을 사용하고 있는지 확인하세요.</div>

In [ ]:
# 모델 구현을 위해 오픈소스 xgboost 알고리즘 설치
%pip install -q xgboost

In [ ]:
# 이 노트북이 SageMaker 작업으로 독립적으로 실행될 수 있도록 mlflow를 설치해야 함.
%pip install --upgrade "mlflow>=2,<3" sagemaker-mlflow

In [ ]:
import pandas as pd
import numpy as np 
import json
import joblib
import xgboost as xgb
import sagemaker
import boto3
import os
import matplotlib.pyplot as plt
import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature
from time import gmtime, strftime, sleep
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

(sagemaker.__version__, boto3.__version__, mlflow.__version__)

In [ ]:
%store -r 

%store

try:
    initialized
except NameError:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN 00-start-here notebook   ")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++")
    user_profile_name = sagemaker.get_execution_role()

In [ ]:
target_col = "y"

In [ ]:
%store target_col

In [ ]:
session = sagemaker.Session()
sm = session.sagemaker_client

## 데이터 로드
다음 셀은 노트북을 [SageMaker 노트북 기반 워크플로](https://docs.aws.amazon.com/sagemaker/latest/dg/notebook-auto-run.html)로 헤드리스 실행할 때 매개변수화를 가능하게 하기 위해 셀 태그로 `parameters`가 태그되어 있습니다. 자세한 내용과 예제는 **노트북을 SageMaker 작업으로 실행** 섹션을 참조하세요. 지금은 무시하세요.

In [ ]:
# 이 셀은 `parameters` 태그가 지정되어 있으며 노트북이 헤드리스로 실행될 경우 덮어쓰여집니다.
file_source = "local"
file_name = "bank-additional-full.csv"
input_path = "./data/bank-additional" 
output_path = "./data"

In [ ]:
# 노트북을 작업으로 실행하거나 비대화형 또는 헤드리스로 실행할 경우, 노트북은 JupyterLab EBS 볼륨에 액세스할 수 없으므로 대신 S3에서 데이터셋을 다운로드합니다.
# 자세한 내용은 "노트북을 SageMaker 작업으로 실행" 섹션을 참조하세요.
if file_source != "local":
    session.download_data(
        path=os.path.join(input_path, ""), 
        bucket=bucket_name,
        key_prefix=f"{bucket_prefix}/input/{file_name}"
    )

## EDA (Explotary Data Analysis)
이 데이터셋에 대해 탐색적 데이터 분석을 해보겠습니다.

In [ ]:
df_data = pd.read_csv(os.path.join(input_path, file_name), sep=";")

pd.set_option("display.max_columns", 500)  # View all of the columns
df_data  # show first 5 and last 5 rows of the dataframe

In [ ]:
# see column metadata
df_data.info()

In [ ]:
# see column statistics
df_data.describe()

In [ ]:
# see target distribution
df_data[target_col].value_counts().plot.bar()

plt.show()

In [ ]:
# see if there are any missing values
df_data.isna().sum()

In [ ]:
cat_columns = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']

fig, axs = plt.subplots(3, 3, sharex=False, sharey=False, figsize=(20, 15))

counter = 0
for cat_column in cat_columns:
    value_counts = df_data[cat_column].value_counts()
    
    trace_x = counter // 3
    trace_y = counter % 3
    x_pos = np.arange(0, len(value_counts))
    
    axs[trace_x, trace_y].bar(x_pos, value_counts.values, tick_label = value_counts.index)
    
    axs[trace_x, trace_y].set_title(cat_column)
    
    for tick in axs[trace_x, trace_y].get_xticklabels():
        tick.set_rotation(90)
    
    counter += 1

plt.show()

In [ ]:
num_columns = ['duration', 'campaign', 'pdays', 'previous']

fig, axs = plt.subplots(2, 2, sharex=False, sharey=False, figsize=(20, 15))

counter = 0
for num_column in num_columns:
    
    trace_x = counter // 2
    trace_y = counter % 2
    
    axs[trace_x, trace_y].hist(df_data[num_column])
    
    axs[trace_x, trace_y].set_title(num_column)
    
    counter += 1

plt.show()

In [ ]:
j_df = pd.DataFrame()

j_df['yes'] = df_data[df_data[target_col] == 'yes']['marital'].value_counts()
j_df['no'] = df_data[df_data[target_col] == 'no']['marital'].value_counts()

j_df.plot.bar(title = 'Marital status and deposit')

## SageMaker와 MLflow 통합을 통한 실험 추적
[MLflow와 함께 Amazon SageMaker를 사용하여 머신러닝 실험을 관리](https://docs.aws.amazon.com/sagemaker/latest/dg/mlflow.html)하고 중앙 관리형 MLflow 서버와 함께 MLflow의 전체 기능을 사용할 수 있습니다.

[MLflow 추적](https://mlflow.org/docs/latest/tracking.html)을 사용하면 반복의 입력, 매개변수, 구성 및 모델을 실험과 실행으로 자동 추적할 수 있습니다. `실행`과 `실험`이 어떻게 구성되는지 이해하려면 [MLflow 개념](https://mlflow.org/docs/latest/tracking.html#concepts)을 참조하세요.

In [ ]:
if file_source == "local":
    # MLflow 서버가 'Created' 또는 'Started' 상태인지 확인
    sm = boto3.client("sagemaker")
    
    while sm.describe_mlflow_tracking_server(TrackingServerName=mlflow_name)['TrackingServerStatus'] not in ['Created', 'Started']:
        print(f"The MLflow server {mlflow_name} is not in the status 'Created' or 'Started'")
        sleep(30)
    else:
        print(f"Using server {mlflow_name}")

In [ ]:
mlflow.set_tracking_uri(mlflow_arn)

In [ ]:
experiment_suffix = strftime('%d-%H-%M-%S', gmtime())
experiment_name = f"from-idea-to-prod-experiment-{experiment_suffix}"
registered_model_name = f"from-idea-to-prod-experiment-model-{experiment_suffix}"

In [ ]:
%store experiment_name

In [ ]:
experiment = mlflow.set_experiment(experiment_name=experiment_name)

## Feature engineering

예시로, 처리 스크립트는 다음과 같은 피처 엔지니어링을 구현합니다:
1. `no_previous_contact`라는 새 열을 생성합니다. `pdays`가 `999`일 때 값을 `1`로 설정하고 그렇지 않으면 `0`으로 설정합니다.
1. `job` 열을 기반으로 고객이 근무 중인지 여부를 보여주는 새 열을 생성합니다.
1. 추론 시간에 피처로 사용하려면 높은 정밀도로 예측해야 하므로 데이터셋에서 경제적 피처를 제거합니다.
1. 통화가 수행되기 전에는 알 수 없으므로 `duration`을 제거합니다.
1. **원-핫 인코딩**을 사용하여 범주형 변수를 숫자로 변환합니다.
1. 타겟 열 `y`를 앞쪽으로 이동합니다.

실제 환경에서는 추가적인 처리, 데이터 품질 처리, 피처 엔지니어링을 구현합니다. 또한 여러 번의 "시도 및 실패" 반복을 거칩니다.

In [ ]:
# pdays가 999 값을 가질 때를 포착하는 Indicator 변수
df_data["no_previous_contact"] = np.where(df_data["pdays"] == 999, 1, 0)

# 적극적으로 고용되지 않은 개인에 대한 Indicator
df_data["not_working"] = np.where(
    np.in1d(df_data["job"], ["student", "retired", "unemployed"]), 1, 0
)

# 불필요한 데이터 제거
df_model_data = df_data.drop(
    ["duration", "emp.var.rate", "cons.price.idx", "cons.conf.idx", "euribor3m", "nr.employed"],
    axis=1,
)


bins = [18, 30, 40, 50, 60, 70, 90]
labels = ['18-29', '30-39', '40-49', '50-59', '60-69', '70-plus']

df_model_data['age_range'] = pd.cut(df_model_data.age, bins, labels=labels, include_lowest=True)
df_model_data = pd.concat([df_model_data, pd.get_dummies(df_model_data['age_range'], prefix='age', dtype=int)], axis=1)
df_model_data.drop('age', axis=1, inplace=True)
df_model_data.drop('age_range', axis=1, inplace=True)

scaled_features = ['pdays', 'previous', 'campaign']
df_model_data[scaled_features] = MinMaxScaler().fit_transform(df_model_data[scaled_features])

df_model_data = pd.get_dummies(df_model_data, dtype=int)  # 범주형 변수를 indicator 집합으로 변환

# "y_no"와 "y_yes"를 단일 레이블 열로 교체하고 앞쪽으로 이동:
df_model_data = pd.concat(
    [
        df_model_data["y_yes"].rename(target_col),
        df_model_data.drop(["y_no", "y_yes"], axis=1),
    ],
    axis=1,
)

In [ ]:
df_model_data

In [ ]:
df_model_data.describe()

## 데이터 분할

[SageMaker XGBoost](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html#InputOutput-XGBoost)는 다음과 같은 libSVM 또는 CSV 형식의 데이터를 기대합니다:

- 첫 번째 열에 타겟 변수
- 헤더 행 없음

In [ ]:
# Shuffle and splitting dataset
train_data, validation_data, test_data = np.split(
    df_model_data.sample(frac=1, random_state=1729),
    [int(0.7 * len(df_model_data)), int(0.9 * len(df_model_data))],
)

print(f"Data split > train:{train_data.shape} | validation:{validation_data.shape} | test:{test_data.shape}")

In [ ]:
# Save data to Studio filesystem
train_data.to_csv(os.path.join(output_path, "train.csv"), index=False, header=False)
validation_data.to_csv(os.path.join(output_path, "validation.csv"), index=False, header=False)
test_data.to_csv(os.path.join(output_path, "test.csv"), index=False, header=False)

## 모델 훈련 및 검증

In [ ]:
train_features = train_data.drop(target_col, axis=1)
train_label = pd.DataFrame(train_data[target_col])

In [ ]:
dtrain = xgb.DMatrix(train_features, label=train_label)

In [ ]:
hyperparams = {
                "max_depth": 5,
                "eta": 0.5,
                "alpha": 2.5,
                "objective": "binary:logistic",
                "subsample" : 0.8,
                "colsample_bytree" : 0.8,
                "min_child_weight" : 3
              }

num_boost_round = 150
nfold = 3
early_stopping_rounds = 10

먼저 훈련 데이터셋의 `nfold`만큼의 폴드에서 모델을 훈련하고 교차 검증을 실행합니다.

In [ ]:
# Cross-validate on training data
cv_results = xgb.cv(
    params=hyperparams,
    dtrain=dtrain,
    num_boost_round=num_boost_round,
    nfold=nfold,
    early_stopping_rounds=early_stopping_rounds,
    metrics=["auc"],
    seed=10,
)

In [ ]:
metrics_data = {
    "binary_classification_metrics": {
        "validation:auc": {
            "value": cv_results.iloc[-1]["test-auc-mean"],
            "standard_deviation": cv_results.iloc[-1]["test-auc-std"]
        },
        "train:auc": {
            "value": cv_results.iloc[-1]["train-auc-mean"],
            "standard_deviation": cv_results.iloc[-1]["train-auc-std"]
        },
    }
}

In [ ]:
print(f"Cross-validated train-auc:{cv_results.iloc[-1]['train-auc-mean']:.2f}")
print(f"Cross-validated validation-auc:{cv_results.iloc[-1]['test-auc-mean']:.2f}")

In [ ]:
cv_results

이제 훈련 데이터셋을 여러 폴드로 분할하는 대신 전체 훈련 데이터셋에서 모델을 재훈련합니다. 조기 중단을 위해 테스트 데이터셋을 사용합니다.

In [ ]:
test_features = test_data.drop(target_col, axis=1)
test_label = pd.DataFrame(test_data[target_col])
dtest = xgb.DMatrix(test_features, label=test_label)

### 실험 실행 생성
[mlflow.start_run()](https://mlflow.org/docs/latest/python_api/mlflow.html#mlflow.start_run) API를 사용하여 새 실행을 생성하고 [log_params()](https://mlflow.org/docs/latest/python_api/mlflow.html#mlflow.log_params) 및 [log_artifact()](https://mlflow.org/docs/latest/python_api/mlflow.html#mlflow.log_artifact) MLflow 로깅 함수를 호출하여 실행에 정보를 기록합니다. mlflow.log_artifact()는 MLflow 서버를 생성할 때 지정한 S3 URI 하위의 MLflow 아티팩트 스토어에 로컬 파일을 업로드합니다.

또한 [log_input()](https://mlflow.org/docs/latest/python_api/mlflow.html#mlflow.log_input) 메서드를 사용하여 데이터셋을 MLflow 아티팩트 스토어에 지속적으로 로그할 수 있습니다.

In [ ]:
run_suffix = strftime('%d-%H-%M-%S', gmtime())

with mlflow.start_run(
    run_name=f"feature-engineering-{run_suffix}",
    description="feature-engineering in the notebook 01 ideation") as run:
    mlflow.log_params(
        {
            "train": 0.7,
            "validate": 0.2,
            "test": 0.1
        }
    )
    # Log input dataset metadata and output
    mlflow.log_artifact(local_path=os.path.join(input_path, file_name))
    mlflow.log_artifact(local_path=os.path.join(output_path, "train.csv"))
    mlflow.log_artifact(local_path=os.path.join(output_path, "validation.csv"))
    mlflow.log_artifact(local_path=os.path.join(output_path, "test.csv"))

### 모델 훈련
[MLflow 모델 Flavor](https://mlflow.org/docs/latest/python_api/index.html#python-api)와 [로깅 함수](https://mlflow.org/docs/latest/tracking/tracking-api.html#tracking-logging-functions)를 사용하여 실험 실행에서 매개변수, 모델, 모델 메트릭 및 다양한 메타데이터를 로그합니다.

In [ ]:
# 프로덕션 코드에서는 고유 ID를 사용해야 합니다.
run_suffix = strftime('%d-%H-%M-%S', gmtime())
max_metric = 0.0
best_model_run_id = 0

with mlflow.start_run(
    run_name=f"training-{run_suffix}",
    description=f"Fit estimator with different max_depth") as parent_run:
    mlflow.set_tags({'mlflow.user':user_profile_name})
    
    # 다양한 max_depth 값에 대해 모델을 훈련합니다.
    for i, d in enumerate([2, 5, 10, 15, 20]):
        hyperparams["max_depth"] = d
        print(f"Fit estimator with max_depth={d}")
    
        with mlflow.start_run(
            run_name=f"max_depth={d}",
            description=f"Fit estimator with max_depth={d}",
            nested=True) as child_run:
            mlflow.set_tags({'mlflow.user':user_profile_name})
            
            mlflow.xgboost.autolog(log_model_signatures=False, log_datasets=False)
            
            # 모델 훈련
            model = xgb.train(
                params=hyperparams, 
                dtrain=dtrain, 
                evals = [(dtrain,'train'), (dtest,'eval')], 
                num_boost_round=num_boost_round, 
                early_stopping_rounds=early_stopping_rounds, 
                verbose_eval = 0
            )
    
            # 메트릭 계산
            test_auc = roc_auc_score(test_label, model.predict(dtest))
            train_auc = roc_auc_score(train_label, model.predict(dtrain))
            
            # 하이퍼파라미터와 메트릭 실행 로깅
            mlflow.log_params(hyperparams)
            mlflow.log_metrics({"test_auc":test_auc, "train_auc":train_auc}, step=i)
    
            if test_auc > max_metric:
                best_model_run_id = child_run.info.run_id
                max_metric = test_auc
    
            print(f"Test AUC: {test_auc:.4f} | Train AUC: {train_auc:.4f}")

### MLflow 모델 레지스트리에 모델 등록
[MLflow 모델 레지스트리](https://mlflow.org/docs/latest/model-registry.html#adding-an-mlflow-model-to-the-model-registry)를 사용하여 모델을 등록합니다.
이 예제에서는 [mlflow.register_model()](https://mlflow.org/docs/latest/python_api/mlflow.html#mlflow.register_model) API를 사용하여 등록된 모델 이름 하에 최고 모델을 새 버전으로 등록합니다. `mlflow.register_model()`은 또한 SageMaker 모델 레지스트리에 모델을 자동으로 등록합니다. 
MLflow 모델을 등록할 때 SageMaker에서 해당 모델 패키지 그룹과 모델 패키지 버전이 생성됩니다.

In [ ]:
model_uri = f"runs:/{best_model_run_id}/xgboost"
mv = mlflow.register_model(model_uri, registered_model_name)

## MLflow UI로 실험 탐색
시작점으로 Studio UI의 **SageMaker Home** > **Experiments** 위젯에서 모든 실험에 액세스할 수 있습니다.

예를 들어, 실험을 선택하세요:

![](img/experiments-studio.png)

실험이 새 브라우저 창의 MLflow UI에서 열립니다:

![](img/experiment-mlflow.png)

분석하고 싶은 실행을 선택하고 **Compare**를 클릭할 수 있습니다. 비교 창에서 실행에 로그된 메트릭과 매개변수를 분석할 수 있습니다:

![](img/comparing-runs.png)

**Models**로 변경하여 등록된 모델과 모든 버전을 확인하세요:

![](img/models-mlflow.png)

MLflow와 함께하는 SageMaker 실험에 대한 더 많은 예제와 세부 정보는 [Amazon SageMaker에서 완전 관리형 MLflow의 일반 가용성 발표](https://aws.amazon.com/blogs/aws/manage-ml-and-generative-ai-experiments-using-amazon-sagemaker-with-mlflow/) 출시 블로그 게시물과 [개발자 가이드](https://docs.aws.amazon.com/sagemaker/latest/dg/mlflow-track-experiments.html)를 참조하세요.

## 선택사항: 노트북을 SageMaker 작업으로 실행
때로는 노트북을 비대화형, 예약된 작업으로 실행하고 싶은 시나리오가 있습니다. Studio는 기존 Amazon EventBridge, SageMaker Training 및 SageMaker Pipelines 서비스로 구축된 빠르고 간단한 도구를 제공하여 노트북 작업을 대화형으로 예약할 수 있도록 도와줍니다. 배포하는 데 추가적인 시간과 비용 오버헤드가 필요할 수 있는 다른 서비스의 기능을 사용하거나 자체 맞춤형 솔루션을 만들 필요가 없습니다.

선택한 일정에 따라 온디맨드로 노트북을 SageMaker 작업으로 실행할 수 있습니다. 또한 여러 노트북을 병렬로 실행하고 노트북의 셀을 매개변수화할 수 있습니다.

### 헤드리스로 실행하도록 노트북 적응
헤드리스 노트북은 Studio 환경 외부의 셸에서 실행됩니다. 따라서 노트북의 코드는 Studio 로컬 스토리지, 환경 변수 또는 Python 스토어에 의존하거나 액세스할 수 없습니다. 로컬 Studio 환경을 사용하는 모든 코드를 그에 따라 변경해야 합니다.

### 실행 방법
이 노트북을 SageMaker 작업으로 비대화형 모드에서 실행하려면 개발자 가이드의 [노트북 기반 워크플로](https://docs.aws.amazon.com/sagemaker/latest/dg/notebook-auto-run.html) 지침을 따르세요:
1. Studio 실행 역할에 대한 신뢰 정책과 추가 IAM 권한을 [구성](https://docs.aws.amazon.com/sagemaker/latest/dg/scheduled-notebook-policies.html)합니다. AWS 사전 프로비저닝된 계정의 도메인에서 이 노트북을 실행하는 경우 필요한 권한이 자동으로 배포됩니다.
2. 아래 지정된 대로 매개변수를 제공합니다.
3. 온디맨드로 노트북을 실행하거나 작업을 예약합니다.
4. 결과를 탐색합니다.

### 매개변수 설정

In [ ]:
# SageMaker에서 사용하는 S3 버킷의 이름을 출력 – bucket_name 매개변수로 이 값이 필요합니다.
print(bucket_name)

In [ ]:
# MLflow 서버 ARN 출력
print(mlflow_arn)

In [ ]:
# 대화형으로 실행하는 경우, 헤드리스 실행을 위해 데이터를 S3에 업로드합니다.
if file_source == 'local':
    input_s3_url = session.upload_data(
        path=os.path.join(input_path, file_name),
        bucket=bucket_name,
        key_prefix=f"{bucket_prefix}/input"
    )
    
    print(input_s3_url)

노트북을 매개변수화하려면 노트북의 단일 셀에 parameters 태그를 [설정](https://docs.aws.amazon.com/sagemaker/latest/dg/notebook-auto-run-troubleshoot-override.html)하여 "매개변수 셀"로 표시합니다. SageMaker 노트북 실행은 런타임에 parameters로 태그된 해당 셀 바로 다음에 새로 생성된 셀을 삽입합니다. 생성된 셀에는 실행 작업을 시작할 때 지정한 값으로 매개변수를 설정하는 코드가 포함됩니다.

노트북 실행 작업은 JupyterLab EBS 볼륨에 액세스할 수 없습니다. 노트북에 전달해야 하는 모든 데이터는 노트북이 액세스할 수 있는 S3 버킷에 복사해야 합니다.

이 노트북을 SageMaker 작업으로 실행하려면 노트북 작업 표시줄에서 **Create a notebook job** 아이콘을 선택하세요:

![](img/notebook-as-sm-job-run.png)

양식의 Parameter 섹션에서 다음 **매개변수**를 지정된 값으로 설정하세요:

![](img/notebook-as-sm-job-parameters.png)

매개변수와 값:

mlflow_arn = <MLflow 서버 ARN으로 설정>
file_source = S3
input_path = /opt/ml/input/data/sagemaker_headless_execution
output_path = /opt/ml/output/data
bucket_name = <SageMaker 버킷 이름으로 설정>
bucket_prefix = from-idea-to-prod/xgboost

**Run now** 또는 **Run on a schedule**을 선택하고 **Create**를 선택하세요.
[SageMaker Python SDK로 프로그래밍 방식으로 노트북 작업을 생성](https://docs.aws.amazon.com/sagemaker/latest/dg/create-notebook-auto-run-sdk.html)할 수도 있습니다.

---

## 2단계로 계속
2단계 [노트북](02-sagemaker-containers.ipynb)을 여세요.

## 실제 프로젝트를 위한 추가 개발 아이디어
- [CatBoost](https://docs.aws.amazon.com/sagemaker/latest/dg/catboost.html), [AutoGluon-Tabular](https://docs.aws.amazon.com/sagemaker/latest/dg/autogluon-tabular.html), 또는 [Linear Learner Algorithm](https://docs.aws.amazon.com/sagemaker/latest/dg/linear-learner.html)과 같은 [SageMaker 내장 알고리즘](https://docs.aws.amazon.com/sagemaker/latest/dg/algos.html) 등 다양한 모델을 시도해보세요.
- [SageMaker Autopilot](https://aws.amazon.com/sagemaker/autopilot/)을 사용하여 최고의 모델을 찾기 위해 다양한 솔루션을 자동으로 탐색해보세요. 이 실습 튜토리얼을 참조하세요: [머신러닝 모델 자동 생성](https://aws.amazon.com/getting-started/hands-on/machine-learning-tutorial-automatically-create-models/)
- [SageMaker Batch Transform](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform.html)을 사용하여 배치 추론을 구현하세요.

## 추가 리소스
- [Build and Train a Machine Learning Model Locally](https://aws.amazon.com/getting-started/hands-on/machine-learning-tutorial-build-model-locally/)
- [Amazon SageMaker XGBoost algorithm](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html)
- [Automatically Create Machine Learning Models](https://aws.amazon.com/getting-started/hands-on/machine-learning-tutorial-automatically-create-models/)
- [Operationalize your Amazon SageMaker Studio notebooks as scheduled notebook jobs](https://aws.amazon.com/blogs/machine-learning/operationalize-your-amazon-sagemaker-studio-notebooks-as-scheduled-notebook-jobs/)
- [Dataset transformations](https://scikit-learn.org/stable/data_transforms.html)
- [Extracting, transforming and selecting features](https://spark.apache.org/docs/latest/ml-features.html)

# kernel 종료

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>